## TP 2 : Exercice CSV
<center>
    <strong>Nom et Prénom:</strong> EL ATTARI Yassine 
</center>

#### Remarque: les chemins des fichiers doit être une URL Hadoop (« hdfs:// », « file:// »), puis nous devons ajouter avant os.getcwd() "file://" dans tous les examples

In [7]:
import time
import pyspark
import os
start  = time.time()
 
spark = pyspark.sql.SparkSession.builder.appName("LoadCsv").getOrCreate()
delimiter = "|"

    #Load 3 csv files into spark dataframe   
person_df = spark.read.options(header='true', inferschema='true',delimiter=delimiter).csv("file://"+os.getcwd()+"/data/csv/person_nodes.csv")
movie_df = spark.read.options(header='true', inferschema='true',delimiter=delimiter).csv("file://"+os.getcwd()+"/data/csv/movie_nodes.csv")
relationships_df = spark.read.options(header='true', inferschema='true',delimiter=delimiter).csv("file://"+os.getcwd()+"/data/csv/acted_in_rels.csv")
#Vérification de la forme des données
header = movie_df.first()
print(header)

Row(id=1, title='The Matrix', tagline='Welcome to the Real World.', poster_image='http://image.tmdb.org/t/p/w185/gynBNzwyaHKtXqlEKKLioNkjKgN.jpg', duration=136, rated='R')


### Au niveau de la jointure, nous pouvons spécifier les champs utilisés pour la jointure comme deuxième paramètre.

In [2]:
    #Prepare a linked dataset of people, movies and the roles for people who played in those movies
df = person_df.join(relationships_df, (person_df.id == relationships_df.person_id))
combined_df = df.join(movie_df, (df.movie_id == movie_df.id))
#Vérification de la forme des données
#print(combined_df[0])

### selon la documentation, pour filter les données on peut utiliser la méthode where ou filter

In [8]:

    #Use where statement analogous to that in Pandas dataframes to find movies associated with name "Tom Hanks"
answer = combined_df.where("name='Tom Hanks'")

#Même résultat
#answer = combined_df.filter("name='Tom Hanks'")


    #Return only actor name, movie title and roles
print (answer.select("name", "title", "roles").show())

+---------+------------+--------------------+
|     name|       title|               roles|
+---------+------------+--------------------+
|Tom Hanks| Cloud Atlas|Old Salty Dog / M...|
|Tom Hanks|Forrest Gump|        Forrest Gump|
+---------+------------+--------------------+

None


In [6]:

    #Save the answer in JSON format 
answer.coalesce(1).select("name", "title", "roles").write.json("file://"+os.getcwd()+"/output/json/result.json")

end = time.time()
print ("Elapsed time: ", (end-start))

Elapsed time:  263.05965638160706
